In [244]:
"""
task: Generate comparison table 
author: Zheng Wei
latest modification date: 2018/5/16
"""

import os  

def get_file(file_dir, start_date, year, week_num, require='WEEKLY'):
    """
    Return a list including closest week_num emails after start_date.2017-10
    """
    temp_list = [files for root, dirs, files in os.walk(file_dir)][0]
    file_list = [(file_name.split('_')[-1].split('.')[0], file_name) for file_name in temp_list 
                 if 'Tier1_monitoring' in file_name and 'xlsx' in file_name]
    # Store 
    emails = []
    if require == 'WEEKLY':
        month, day = start_date.split('/')
        if len(month) == 1:
            month = '0' + month
        if len(day) == 1:
            day = '0' + day
        start_date = int(str(year) + month + day) 
        for date, name in file_list:
            try:
                date = int(date)
                if date > start_date:
                    emails.append((calc_interval(str(date), str(start_date)), name))
            except ValueError:
                continue
    elif require == 'MONTHLY':
        year = int(start_date.split('-')[0])
        month = int(start_date.split('-')[1])
        for date, name in file_list:
            try:
                int(date)
                year1 = int(date[:4])
                month1 = int(date[4:6])
                day1 = int(date[6:])
                if year1 > year:
                    if month1 == 1:
                        emails.append(((12-month)*30 + day1, name))
                    elif month1 == 2:
                        emails.append(((12-month)*30 + 31 + day1, name))
                    elif month1 == 3:
                        emails.append(((12-month)*30 + 59 + day1, name)) 
                    else:
                        emails.append((1e8, name))
                elif year1 == year and month1 > month:
                    emails.append(((month1-month-1)*30 + day1, name))
                else:
                    continue
                if date > start_date:
                    emails.append((calc_interval(str(date), str(start_date)), name))
            except ValueError:
                continue
    else:
        raise ValueError("no existence require(only 'WEEKLY' and MONTHLY)")
    return sorted(emails, key = lambda x:x[0])[:week_num]

In [236]:
path = 'C:/Users/zhewei/Desktop/cam_weekly_report/'
top10_emails = get_file(path, '2017-10', year, columns+3, 'MONTHLY')
print(top10_emails)


[(6, 'cam_monitor_report_20171106.xlsx'), (13, 'cam_monitor_report_20171113.xlsx'), (20, 'cam_monitor_report_20171120.xlsx'), (26, 'cam_monitor_report_20171126.xlsx'), (33, 'cam_monitor_report_20171203.xlsx'), (40, 'cam_monitor_report_20171210.xlsx'), (48, 'cam_monitor_report_20171218.xlsx'), (55, 'cam_monitor_report_20171225.xlsx'), (61, 'cam_monitor_report_20180101.xlsx'), (82, 'cam_monitor_report_20180122.xlsx'), (89, 'cam_monitor_report_20180129.xlsx'), (96, 'cam_monitor_report_20180205.xlsx'), (117, 'cam_monitor_report_20180226.xlsx')]


<function print>

In [38]:
def calc_interval(date1, date2, require):
    """
    Calculate the interval between two date.
    """
    interval = 0
    year1, month1, day1 = date1[:4], date1[4:6], date1[6:]
    year2, month2, day2 = date2[:4], date2[4:6], date2[6:]
    if year1 != year2:
        if month1 == '01':
            interval = int(day1)
            if month2 == '10':
                interval += (31 + 30 + 31 - int(day2))
            elif month2 == '11':
                interval += (31 + 30 - int(day2))
            elif month2 == '12':
                interval += (31 - int(day2))
            else:
                return 1e8
        elif month1 == '02':
            interval = 31 + int(day1)
            if month2 == '11':
                interval += (31 + 30 - int(day2))
            elif month2 == '12':
                interval += (31 - int(day2))
            else:
                return 1e8
        elif month1== '03':
            interval = 31 + 28 + int(day1)
            if month2 == '11':
                interval += (31 + 30 - int(day2))
            elif month2 == '12':
                interval += (31 - int(day2))
            else:
                return 1e8
        else:
            return 1e8
    else:
        interval = (int(month1) - int(month2))*30 - int(day2) + int(day1)
    return interval

In [233]:
"""---------------------------------Grap acceptable value from reports (WEEkLY) ------------------------------------"""

import pandas as pd

path = 'C:/Users/zhewei/Desktop/atom_weekly_report/'
init_date = '9/25'
year = 2017
columns = 10
rows = 12

result = {init_date: ['#']*columns}
while 1:
    print(result)
    temp_date = list(result.keys())[-1]
    top_emails = get_file(path, temp_date, year, columns+3)
    for interval, name in top_emails:
        # Weekly, Seg2 Weekly
        temp_data = pd.read_excel(path + name, sheet_name='Seg2 Weekly')
        try:
            if (interval-1)//7 < columns:
                # CAM17_V2_EMS_SCORE, ATOM_2017_Score_2
                temp_value = list(temp_data[temp_date])[list(temp_data['Week']).index('ATOM_2017_Score_2')]
                if '.' in str(temp_value):
                    result[temp_date][(interval - 1)//7] = "%.1f%%" % (temp_value * 100)
            header = list(temp_data.columns)
            next_date = header[header.index(temp_date)+1]
            if isinstance(next_date, str) and '/' in next_date and next_date not in result:
                if int(next_date.split('/')[0]) < int(temp_date.split('/')[0]):
                    year += 1
                result[next_date] = ['#']*columns
        except KeyError:
            continue
    if len(result) > rows:
        del result[next_date]
        break

{'9/25': ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']}
{'9/25': ['#', '87.3%', '89.2%', '89.5%', '87.0%', '86.1%', '86.8%', '86.8%', '86.9%', '86.9%'], '10/2': ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']}
{'9/25': ['#', '87.3%', '89.2%', '89.5%', '87.0%', '86.1%', '86.8%', '86.8%', '86.9%', '86.9%'], '10/2': ['92.2%', '89.9%', '91.3%', '91.4%', '91.1%', '90.7%', '90.8%', '90.8%', '91.1%', '91.2%'], '10/9': ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']}
{'9/25': ['#', '87.3%', '89.2%', '89.5%', '87.0%', '86.1%', '86.8%', '86.8%', '86.9%', '86.9%'], '10/2': ['92.2%', '89.9%', '91.3%', '91.4%', '91.1%', '90.7%', '90.8%', '90.8%', '91.1%', '91.2%'], '10/9': ['86.6%', '87.5%', '87.5%', '87.6%', '87.6%', '88.0%', '88.0%', '88.1%', '88.1%', '88.1%'], '10/16': ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']}
{'9/25': ['#', '87.3%', '89.2%', '89.5%', '87.0%', '86.1%', '86.8%', '86.8%', '86.9%', '86.9%'], '10/2': ['92.2%', '89.9%', '91.3%', '91.4%', '91.1%', '90.7%', '90.8%'

In [249]:
"""---------------------------------Grap acceptable value from reports (MONTHLY) ------------------------------------"""

import pandas as pd

path = 'C:/Users/zhewei/Desktop/tier1_weekly_report/'
init_date = '2017-10'
year = 2017
columns = 10
rows = 2

# Tier
loc = [2, 10, 18]

result = {init_date: ['#']*columns}
while 1:
    print(result)
    temp_date = list(result.keys())[-1]
    top_emails = get_file(path, temp_date, year, columns+3, 'MONTHLY')
    for interval, name in top_emails:
        temp_data = pd.read_excel(path + name, sheet_name='Summary')
        try:
            head_line = ['-'.join(str(item).split('-')[:2]) for item in list(a.iloc[0])]
            # print(head_line, name)
            if (interval-1)//7 < 10:
                # Tier1
                # head_line = ['-'.join(str(item).split('-')[:2]) for item in list(a.iloc[0])]
                if temp_date in head_line:
                    temp_value = list(temp_data.iloc[10])[head_line.index(temp_date)]
                else:
                    continue
                # temp_value = list(temp_data[temp_date])[list(temp_data['Month']).index('ATOM_2017_Score_2')]
                if '.' in str(temp_value):
                    result[temp_date][(interval - 1)//7] = "%.1f%%" % (temp_value * 100)
            # header = list(temp_data.columns)
            next_date = header[header.index(temp_date)+1]
            if isinstance(next_date, str) and '-' in next_date and next_date not in result:
#                 if int(next_date.split('-')[0]) > int(temp_date.split('-')[0]):
#                     year += 1
                result[next_date] = ['#']*columns
        except KeyError or IndexError:
            continue
    if len(result) > rows:
        del result[next_date]
        break

{'2017-10': ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']}
{'2017-10': ['96.6%', '96.6%', '96.5%', '96.5%', '96.5%', '96.5%', '96.6%', '96.5%', '96.5%', '96.5%'], '2017-11': ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']}


In [250]:
output = pd.DataFrame(result, index = ['1st week', '2nd week', '3rd week', '4th week', '5th week', 
                                       '6th week', '7th week', '8th week', '9th week','10th week'])

In [251]:
output.T

,1st week,2nd week,3rd week,4th week,5th week,6th week,7th week,8th week,9th week,10th week
2017-10,96.6%,96.6%,96.5%,96.5%,96.5%,96.5%,96.6%,96.5%,96.5%,96.5%
2017-11,96.0%,96.0%,96.0%,96.0%,96.0%,96.0%,#,96.1%,96.2%,96.2%


In [252]:
writer = pd.ExcelWriter("C:/Users/zhewei/Desktop/Task/ATO_month.xlsx")
#df1 = pd.DataFrame({'col1':[1,1], 'col2':[2,2]})
output.T.to_excel(writer,'Monthly')
writer.save()